# Multistage Stochastic Unit Commitment Problem

In [90]:
import os
import numpy as np
import pandas as pd
import gurobipy as gp
from scipy import stats, linalg
from itertools import product


from sddip import config, utils, tree


## Parameters

In [91]:
test_case_raw_dir = "WB2/raw"

test_case_raw_dir = os.path.join(config.test_cases_dir, test_case_raw_dir)

bus_file_raw = os.path.join(test_case_raw_dir, "bus_data.txt")
branch_file_raw = os.path.join(test_case_raw_dir, "branch_data.txt")
gen_file_raw = os.path.join(test_case_raw_dir, "gen_data.txt")
gen_cost_file_raw = os.path.join(test_case_raw_dir, "gen_cost_data.txt")
ren_data_file = os.path.join(test_case_raw_dir, "ren_data.txt")
storage_data_file = os.path.join(test_case_raw_dir, "storage_data.txt")
scenario_data_file = os.path.join(test_case_raw_dir, "scenario_data.txt")


bus_df = pd.read_csv(bus_file_raw, delimiter="\s+")
branch_df = pd.read_csv(branch_file_raw, delimiter="\s+")
gen_df = pd.read_csv(gen_file_raw, delimiter="\s+")
gen_cost_df = pd.read_csv(gen_cost_file_raw, delimiter="\s+")
ren_df = pd.read_csv(ren_data_file, delimiter="\s+")
storage_df = pd.read_csv(storage_data_file, delimiter="\s+")

scenario_df = pd.read_csv(scenario_data_file, delimiter="\s+")

In [92]:
nodes = bus_df.bus_i.values.tolist()
edges = branch_df[["fbus", "tbus"]].values.tolist()

graph = utils.Graph(nodes, edges)

ref_bus = bus_df.loc[bus_df.type == 3].bus_i.values[0]

a_inc = graph.incidence_matrix()
b_l = (-branch_df.x /(branch_df.r**2 + branch_df.x**2)).tolist()
b_diag = np.diag(b_l)

m1 = b_diag.dot(a_inc)
m2 = a_inc.T.dot(b_diag).dot(a_inc)

m1 = np.delete(m1, ref_bus-1, 1)
m2 = np.delete(m2, ref_bus-1, 0)
m2 = np.delete(m2, ref_bus-1, 1)

ptdf = m1.dot(np.linalg.inv(m2))

ptdf = np.insert(ptdf, ref_bus-1, 0, axis=1)

In [93]:
########################################################################################################################
# Deterministic parameters
########################################################################################################################
gc = np.array(gen_cost_df.c1)
suc = np.array(gen_cost_df.startup)
sdc = np.array( gen_cost_df.startup)
pg_min = np.array(gen_df.Pmin)
pg_max = np.array(gen_df.Pmax)
pl_max = np.array(branch_df.rateA)

n_gens = len(gc)
n_lines, n_buses = ptdf.shape

# Lists of generators at each bus
#
# Example: [[0,1], [], [2]]
# Generator 1 & 2 are located at bus 1
# No Generator is located at bus 2
# Generator 3 is located at bus 3
gens_at_bus = [[] for _ in range(n_buses)]
g = 0
for b in gen_df.bus.values:
    gens_at_bus[b-1].append(g)
    g+=1

# TODO Add ramp rate limits
rg_up_max = np.full(n_gens, 100)
rg_down_max = np.full(n_gens, 100)

# TODO Adjust penalty
penalty = 1000
    
########################################################################################################################
# Stochastic parameters
########################################################################################################################
n_realizations_per_stage = scenario_df.groupby("t")["n"].nunique().tolist()
n_stages = len(n_realizations_per_stage)

prob = []
p_d = []
re = []

for t in range(n_stages):
    stage_df = scenario_df[scenario_df["t"] == t+1]
    p_d.append(stage_df[scenario_df.columns[scenario_df.columns.to_series().str.contains('Pd')]].values.tolist())
    re.append(stage_df[scenario_df.columns[scenario_df.columns.to_series().str.contains('Re')]].values.tolist())
    prob.append(stage_df["p"].values.tolist())


########################################################################################################################
# Expected values of stochastic parameters
########################################################################################################################
ex_pd = [np.array(prob[t]).dot(np.array(p_d[t])) for t in range(n_stages)]

In [94]:
storage_buses = storage_df["bus"].values.tolist()
n_storages = len(storage_buses)
storages_at_bus = [[] for _ in range(n_buses)]

s = 0
for b in storage_buses:
    storages_at_bus[b-1].append(s)
    s+=1

rc_max = storage_df["Rc"].values.tolist()
rdc_max = storage_df["Rdc"].values.tolist()
soc_max = storage_df["SOC"].values.tolist()

eff_c = storage_df["Effc"].values.tolist()
eff_dc = storage_df["Effdc"].values.tolist()

In [95]:
storage_buses

[1]

In [96]:
# TODO min up-/down-time
min_up_time = [1]*n_gens
min_down_time = [1]*n_gens

In [97]:
gens_at_bus

[[0], []]

In [98]:
# prob[t][n]
# Probability of realization n at stage t
#prob

In [99]:
# p_d[t][n][b]
# Demand in stage t and realization n at bus b
#p_d

In [100]:
# ex_pd[t][b]
# Expected demand in stage t at bus b
#ex_pd

In [101]:
scenario_tree = tree.ScenarioTree(n_realizations_per_stage)
print(scenario_tree)

ScenarioTree: Stages = 3, Nodes = 7


## Optimization

### Variables

In [102]:
model = gp.Model("MSUC")

x = {}
y = {}
s_up = {}
s_down = {}
ys_p = {}
ys_n = {}
ys_charge = {}
ys_discharge = {}
u = {}
soc = {}

# TODO Set startup and shutdown costs
# suc = [10]*n_gens
# sdc = [10]*n_gens

for t in range(n_stages):
    for node in scenario_tree.get_stage_nodes(t):
        n = node.index
        for g in range(n_gens):
            x[t,n,g] = model.addVar(vtype = gp.GRB.BINARY, name = f"x_{t+1}_{n+1}_{g+1}")
            y[t,n,g] = model.addVar(vtype = gp.GRB.CONTINUOUS, lb = 0, name = f"y_{t+1}_{n+1}_{g+1}")
            s_up[t,n,g] = model.addVar(vtype = gp.GRB.BINARY, name = f"s_up_{t+1}_{n+1}_{g+1}")
            s_down[t,n,g] = model.addVar(vtype = gp.GRB.BINARY, name = f"s_down_{t+1}_{n+1}_{g+1}")
        for s in range(n_storages):
            ys_charge[t,n,s] = model.addVar(vtype = gp.GRB.CONTINUOUS, lb = 0, name = f"ys_c_{t+1}_{n+1}_{s+1}")
            ys_discharge[t,n,s] = model.addVar(vtype = gp.GRB.CONTINUOUS, lb = 0, name = f"ys_d_{t+1}_{n+1}_{s+1}")
            u[t,n,s] = model.addVar(vtype = gp.GRB.BINARY, name = f"u_{t+1}_{n+1}_{s+1}")
            soc[t,n,s] = model.addVar(vtype = gp.GRB.CONTINUOUS, lb = 0, name = f"soc_{t+1}_{n+1}_{s+1}")
        ys_p[t,n] = model.addVar(vtype = gp.GRB.CONTINUOUS, lb = 0, name = f"ys_p_{t+1}_{n+1}")
        ys_n[t,n] = model.addVar(vtype = gp.GRB.CONTINUOUS, lb = 0, name = f"ys_n_{t+1}_{n+1}")

model.update()

### Constraints

In [103]:
# Objective
conditional_probabilities = []
p=1
for n in range(scenario_tree.n_stages):
    p = p*1/n_realizations_per_stage[n]
    conditional_probabilities.append(p)

obj = gp.quicksum(conditional_probabilities[t]*(gc[g]*y[t,n,g] + suc[g]*s_up[t,n,g] + sdc[g]*s_down[t,n,g] + penalty*(ys_p[t,n] + ys_n[t,n])) 
                    for t in range(n_stages)
                    for n in range(scenario_tree.n_nodes_per_stage[t])
                    for g in range(n_gens))

model.setObjective(obj)


# Balance constraints
model.addConstrs((gp.quicksum(y[t,n.index,g] for g in range(n_gens)) 
                    + gp.quicksum(eff_dc[s]*ys_discharge[t,n.index,s] - ys_charge[t,n.index,s] for s in range(n_storages)) 
                    + ys_p[t,n.index] - ys_n[t,n.index] 
                    == gp.quicksum(p_d[t][n.realization])-gp.quicksum(re[t][n.realization])
                    for t in range(n_stages)
                    for n in scenario_tree.get_stage_nodes(t)),                    
                    "balance")


# Generator constraints
model.addConstrs((y[t,n,g] >= pg_min[g]*x[t,n,g] 
                    for g in range(n_gens)
                    for t in range(n_stages) 
                    for n in range(scenario_tree.n_nodes_per_stage[t])),
                    "min-generation")

model.addConstrs((y[t,n,g] <= pg_max[g]*x[t,n,g]  
                    for g in range(n_gens)
                    for t in range(n_stages) 
                    for n in range(scenario_tree.n_nodes_per_stage[t])),
                    "max-generation")


# Storage constraints
model.addConstrs((ys_charge[t,n,s] <= rc_max[s]*u[t,n,s]
                    for s in range(n_storages)
                    for t in range(n_stages)
                    for n in range(scenario_tree.n_nodes_per_stage[t])),
                    "max-charge-rate")

model.addConstrs((ys_discharge[t,n,s] <= rdc_max[s]*(1-u[t,n,s])
                    for s in range(n_storages)
                    for t in range(n_stages)
                    for n in range(scenario_tree.n_nodes_per_stage[t])),
                    "max-discharge-rate")

model.addConstrs((soc[t,n,s] <= soc_max[s]
                    for s in range(n_storages)
                    for t in range(n_stages)
                    for n in range(scenario_tree.n_nodes_per_stage[t])),
                    "max-soc")

# SOC transfer
# t=0
soc_init = [0.5*s for s in soc_max]
model.addConstrs((soc[0,0,s] == soc_init[s] + eff_c[s]*ys_charge[0,0,s] - ys_discharge[0,0,s] 
                    for s in range(n_storages)),
                    "soc")
# t>0
for t in range(1,n_stages):
    for node in scenario_tree.get_stage_nodes(t):
        n = node.index
        a_n = node.parent.index
        model.addConstrs((soc[t,n,s] == soc[t-1, a_n, s] + eff_c[s]*ys_charge[t,n,s] - ys_discharge[t,n,s] 
                            for s in range(n_storages)),
                            "soc")


# Power flow constraints
for t in range(n_stages):
    for node in scenario_tree.get_stage_nodes(t):
        n = node.index
        line_flows = [gp.quicksum(ptdf[l,b] * (gp.quicksum(y[t,n,g] for g in gens_at_bus[b]) 
                        + gp.quicksum(eff_dc[s]*ys_discharge[t,n,s] - ys_charge[t,n,s] for s in storages_at_bus[b]) 
                        - p_d[t][node.realization][b] + re[t][node.realization][b])
                        for b in range(n_buses)) for l in range(n_lines)]
        model.addConstrs((line_flows[l] <= pl_max[l] for l in range(n_lines)), "power-flow(1)")
        model.addConstrs((-line_flows[l] <= pl_max[l] for l in range(n_lines)), "power-flow(2)")


# Startup shutdown constraints
# t=0
x_init = [0]*n_gens
model.addConstrs((x[0,0,g] - x_init[g] <= s_up[0,0,g] for g in range(n_gens)), "up-down(1)")
model.addConstrs((x[0,0,g] - x_init[g] == s_up[0,0,g] - s_down[0,0,g] for g in range(n_gens)), "up-down(2)")
# t>0
for t in range(1,n_stages):
    for node in scenario_tree.get_stage_nodes(t):
        n = node.index
        a_n = node.parent.index
        model.addConstrs((x[t,n,g] - x[t-1,a_n,g] <= s_up[t,n,g] 
                        for g in range(n_gens)), 
                        "up-down(1)")
        model.addConstrs((x[t,n,g] - x[t-1,a_n,g] == s_up[t,n,g] - s_down[t,n,g] 
                        for g in range(n_gens)), 
                        "up-down(2)")


# Ramp rate constraints
# t=0
y_init = [0]*n_gens
model.addConstrs((y[0,0,g] - y_init[g] <= rg_up_max[g] for g in range(n_gens)), "rate-up")
model.addConstrs((y_init[g] - y[0,0,g] <= rg_down_max[g] for g in range(n_gens)), "rate-down(2)")
# t>0
for t in range(1,n_stages):
    for node in scenario_tree.get_stage_nodes(t):
        n = node.index
        a_n = node.parent.index
        model.addConstrs((y[t,n,g] - y[t-1,a_n,g] <= rg_up_max[g]
                        for g in range(n_gens)), 
                        "rate-up")
        model.addConstrs((y[t-1,a_n,g] - y[t,n,g] <= rg_down_max[g]
                        for g in range(n_gens)), 
                        "rate-down")


# Minimum up- and down-time constraints
for g in range(n_gens):
    for t in range(1, min_up_time[g]):
        for node in scenario_tree.get_stage_nodes(t):
            n = node.index
            ancestors = node.get_ancestors()
            model.addConstr((gp.quicksum(x[m.stage,m.index,g] for m in ancestors) >= (t+1)*s_down[t,n,g]), "min-uptime")
    
    for t in range(min_up_time[g], n_stages):
        for node in scenario_tree.get_stage_nodes(t):
            n = node.index
            ancestors = node.get_ancestors(min_up_time[g])
            model.addConstr((gp.quicksum(x[m.stage, m.index,g] for m in ancestors) >= min_up_time[g]*s_down[t,n,g]), "min-uptime")

    for t in range(1, min_down_time[g]):
        for node in scenario_tree.get_stage_nodes(t):
            n = node.index
            ancestors = node.get_ancestors()
            model.addConstr((gp.quicksum((1-x[m.stage,m.index,g]) for m in ancestors) >= (t+1)*s_up[t,n,g]), "min-downtime")

    for t in range(min_down_time[g], n_stages):
        for node in scenario_tree.get_stage_nodes(t):
            n = node.index
            ancestors = node.get_ancestors(min_down_time[g])
            model.addConstr((gp.quicksum((1-x[m.stage,m.index,g]) for m in ancestors) >= min_down_time[g]*s_up[t,n,g]), "min-downtime")


model.update()
#model.display()

### Solve

In [104]:
model.setParam("OutputFlag",0)

model.optimize()

#model.setParam("OutputFlag",1)

model.printAttr("X")
print()
print(f"Optimal value: {obj.getValue()}")


Optimal value: 882.8050643276322


In [105]:
display_results = True

if display_results:
    x_out = [f"x[{t+1},{n+1},{g+1}]:  {x[t,n,g].x}" for t in range(n_stages) for n in range(scenario_tree.n_nodes_per_stage[t]) for g in range(n_gens)]
    y_out = [f"y[{t+1},{n+1},{g+1}]:  {y[t,n,g].x}" for t in range(n_stages) for n in range(scenario_tree.n_nodes_per_stage[t]) for g in range(n_gens)]
    s_up_out = [f"s_up[{t+1},{n+1},{g+1}]:  {s_up[t,n,g].x}" for t in range(n_stages) for n in range(scenario_tree.n_nodes_per_stage[t]) for g in range(n_gens)]
    s_down_out = [f"s_down[{t+1},{n+1},{g+1}]:  {s_down[t,n,g].x}" for t in range(n_stages) for n in range(scenario_tree.n_nodes_per_stage[t]) for g in range(n_gens)]
    soc_out = [f"soc[{t+1},{n+1},{s+1}]:  {soc[t,n,s].x}" for t in range(n_stages) for n in range(scenario_tree.n_nodes_per_stage[t]) for s in range(n_storages)]
    ys_charge_out = [f"y_c[{t+1},{n+1},{s+1}]:  {ys_charge[t,n,s].x}" for t in range(n_stages) for n in range(scenario_tree.n_nodes_per_stage[t]) for s in range(n_storages)]
    ys_discharge_out = [f"y_dc[{t+1},{n+1},{s+1}]:  {ys_discharge[t,n,s].x}" for t in range(n_stages) for n in range(scenario_tree.n_nodes_per_stage[t]) for s in range(n_storages)]
    ys_p_out =  [f"ys_p[{t+1},{n+1}]:  {ys_p[t,n].x}" for t in range(n_stages) for n in range(scenario_tree.n_nodes_per_stage[t])]
    ys_n_out = [f"ys_n[{t+1},{n+1}]:  {ys_n[t,n].x}" for t in range(n_stages) for n in range(scenario_tree.n_nodes_per_stage[t])]

    print("Commitment decisions")
    for text in x_out:
        print(f"{text}")

    print("Dispatch decisions")
    for text in y_out:
        print(f"{text}")

    print("Startup decisions")
    for text in s_up_out:
        print(f"{text}")

    print("Shutdown decisions")
    for text in s_down_out:
        print(f"{text}")

    print("State of charge")
    for text in soc_out:
        print(f"{text}")

    print("Charge/discharge")
    for text in ys_charge_out:
        print(f"{text}")
    for text in ys_discharge_out:
        print(f"{text}")

    print("Slack variables")
    for t1, t2 in zip(ys_p_out, ys_n_out):
        print(t1)
        print(t2)

Commitment decisions
x[1,1,1]:  1.0
x[2,1,1]:  1.0
x[2,2,1]:  1.0
x[3,1,1]:  1.0
x[3,2,1]:  1.0
x[3,3,1]:  1.0
x[3,4,1]:  1.0
Dispatch decisions
y[1,1,1]:  59.75322523013091
y[2,1,1]:  159.7532252301309
y[2,2,1]:  138.77399135017043
y[3,1,1]:  225.99740593689847
y[3,2,1]:  259.7532252301309
y[3,3,1]:  205.01817205693806
y[3,4,1]:  238.77399135017043
Startup decisions
s_up[1,1,1]:  1.0
s_up[2,1,1]:  0.0
s_up[2,2,1]:  0.0
s_up[3,1,1]:  -0.0
s_up[3,2,1]:  -0.0
s_up[3,3,1]:  -0.0
s_up[3,4,1]:  -0.0
Shutdown decisions
s_down[1,1,1]:  0.0
s_down[2,1,1]:  0.0
s_down[2,2,1]:  0.0
s_down[3,1,1]:  0.0
s_down[3,2,1]:  0.0
s_down[3,3,1]:  0.0
s_down[3,4,1]:  0.0
State of charge
soc[1,1,1]:  60.85862907602581
soc[2,1,1]:  54.02076612003958
soc[2,2,1]:  89.35751545804729
soc[3,1,1]:  0.0
soc[3,2,1]:  0.0
soc[3,3,1]:  14.357515458047288
soc[3,4,1]:  14.357515458047288
Charge/discharge
y_c[1,1,1]:  0.0
y_c[2,1,1]:  0.0
y_c[2,2,1]:  28.49888638202148
y_c[3,1,1]:  0.0
y_c[3,2,1]:  0.0
y_c[3,3,1]:  0.0
y

In [106]:
conditional_probabilities = []
p=1
for n in range(scenario_tree.n_stages):
    p = p*1/n_realizations_per_stage[n]
    conditional_probabilities.append(p)

costs = []
c = 0
for t in reversed(range(n_stages)):
    for n in range(scenario_tree.n_nodes_per_stage[t]):
        for g in range(n_gens):
            c += conditional_probabilities[t] * (y[t,n,g].x*gc[g] + s_up[t,n,g].x*suc[g] + s_down[t,n,g].x*sdc[g]+ penalty*(ys_p[t,n].x+ys_n[t,n].x))
    costs.append(c)


costs

[464.7713972870689, 763.2986138673702, 882.8050643276321]

In [107]:
model.setParam("OutputFlag", 1)
model.display()

Set parameter OutputFlag to value 1
Minimize
<gurobi.LinExpr: 2.0 y_1_1_1 + 1000.0 ys_p_1_1 + 1000.0 ys_n_1_1 + y_2_1_1
+ 500.0 ys_p_2_1 + 500.0 ys_n_2_1 + y_2_2_1 + 500.0 ys_p_2_2 + 500.0 ys_n_2_2
+ 0.5 y_3_1_1 + 250.0 ys_p_3_1 + 250.0 ys_n_3_1 + 0.5 y_3_2_1 + 250.0 ys_p_3_2
+ 250.0 ys_n_3_2 + 0.5 y_3_3_1 + 250.0 ys_p_3_3 + 250.0 ys_n_3_3 + 0.5 y_3_4_1
+ 250.0 ys_p_3_4 + 250.0 ys_n_3_4>
Subject To
balance[0,<sddip.tree.Node object at 0x000001EB6A492FD0>]: <gurobi.LinExpr: y_1_1_1 +
 -1.0 ys_c_1_1_1 + ys_d_1_1_1 + ys_p_1_1 + -1.0 ys_n_1_1> = 73.8946
balance[1,<sddip.tree.Node object at 0x000001EB6A4921C0>]: <gurobi.LinExpr: y_2_1_1 +
 -1.0 ys_c_2_1_1 + ys_d_2_1_1 + ys_p_2_1 + -1.0 ys_n_2_1> = 166.591
balance[1,<sddip.tree.Node object at 0x000001EB6A492580>]: <gurobi.LinExpr: y_2_2_1 +
 -1.0 ys_c_2_2_1 + ys_d_2_2_1 + ys_p_2_2 + -1.0 ys_n_2_2> = 110.275
balance[2,<sddip.tree.Node object at 0x000001EB6A492DF0>]: <gurobi.LinExpr: y_3_1_1 +
 -1.0 ys_c_3_1_1 + ys_d_3_1_1 + ys_p_3_1 + -1.0 ys